In [2]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np
import pandas as pd

import sys
import os

import scipy.signal
from ecgdetectors import Detectors

import math
from DataHandlers.DiagEnum import DiagEnum
import DataHandlers.DiagEnum
import DataHandlers.SAFERDataset as SAFERDataset
import DataHandlers.CinC2020Dataset as CinC2020Dataset
import DataHandlers.CinC2020Enums
import importlib
import DataHandlers.CinCDataset as CinCDataset
import DataHandlers.DataAugmentations as DataAugmentations
from multiprocesspandas import applyparallel
importlib.reload(SAFERDataset)
importlib.reload(CinC2020Dataset)

import DataHandlers.DataProcessUtilities
importlib.reload(DataHandlers.DataProcessUtilities)
from DataHandlers.DataProcessUtilities import *
import Utilities.Plotting
importlib.reload(Utilities.Plotting)
from Utilities.Plotting import *

# A fudge because I moved the files
sys.modules["SAFERDataset"] = SAFERDataset
sys.modules["CinC2020Dataset"] = CinC2020Dataset
sys.modules["DiagEnum"] = DataHandlers.DiagEnum
sys.modules["CinC2020Enums"] = DataHandlers.CinC2020Enums
sys.modules["CinCDataset"] = CinCDataset

In [5]:
def prepare_safer_data(pt_data, ecg_data):
    if "length" in ecg_data:
        ecg_data = ecg_data[ecg_data["length"] == 9120]

    ecg_data = ecg_data[ecg_data["measDiag"] != DiagEnum.PoorQuality]
    # ecg_data = ecg_data[ecg_data["tag_orig_Poor_Quality"] == 0]

    ecg_data = ecg_data[ecg_data["rri_len"] > 5]


    pt_data.index = pt_data["ptID"]
    ecg_data_new = SAFERDataset.generate_af_class_labels(ecg_data)
    pt_data = SAFERDataset.add_ecg_class_counts(pt_data, ecg_data)

    return pt_data, ecg_data_new

pt_data = SAFERDataset.load_pt_dataset(1)
ecg_data = SAFERDataset.load_ecg_csv(1, pt_data, ecg_range=None, ecg_meas_diag=None, feas2_offset=10000, feas2_ecg_offset=200000)

ecg_data["feas"] = 1
ecg_data["length"] = 9120
ecg_data["rri_len"] = 20

pt_data, ecg_data_new = prepare_safer_data(pt_data, ecg_data)
# train_pts, test_pts, val_pts = generate_patient_splits(pt_data, 0.15, 0.15)

In [8]:
ecg_data[(~ecg_data["measID"].isin(ecg_data_new["measID"])) & (ecg_data["ptDiag"] == DiagEnum.AF)]

,ptID,age,ptDiag,ptDiagRev1,ptDiagRev2,ptDiagRev3,cardRev,measDiag,measDiagRev1,measDiagRev2,...,unlikelyAF,perhapsAF,measID,data,adc_gain,file_path,class_index,length,feas,rri_len
measID,,,,,,,,,,,,,,,,,,,,,
1437,17,76.0,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,1437,None,None,ECGs/001000/saferF1_001437,0,9120,1,20
1464,17,76.0,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,1,DiagEnum.AF,DiagEnum.NoAF,DiagEnum.AF,...,0,1,1464,None,None,ECGs/001000/saferF1_001464,0,9120,1,20
2770,35,72.0,DiagEnum.AF,DiagEnum.CannotExcludePathology,DiagEnum.NoAF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,2770,None,None,ECGs/002000/saferF1_002770,0,9120,1,20
2773,35,72.0,DiagEnum.AF,DiagEnum.CannotExcludePathology,DiagEnum.NoAF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,2773,None,None,ECGs/002000/saferF1_002773,0,9120,1,20
2776,35,72.0,DiagEnum.AF,DiagEnum.CannotExcludePathology,DiagEnum.NoAF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,2776,None,None,ECGs/002000/saferF1_002776,0,9120,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162089,2134,68.0,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,162089,None,None,ECGs/162000/saferF1_162089,0,9120,1,20
162090,2134,68.0,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,162090,None,None,ECGs/162000/saferF1_162090,0,9120,1,20
162091,2134,68.0,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,DiagEnum.AF,1,DiagEnum.Undecided,DiagEnum.Undecided,DiagEnum.Undecided,...,0,1,162091,None,None,ECGs/162000/saferF1_162091,0,9120,1,20


In [10]:
feas1_path = r"D:\2022_23_DSiromani\Feas1"
feas1_noise_predictions = pd.read_pickle(os.path.join(feas1_path, "ECGs/feas1_noise_predictions.pk"))

In [16]:
(feas1_noise_predictions > 0).value_counts()

False    129299
True      23933
dtype: int64

In [18]:
ecg_data["noise_prediction"] = feas1_noise_predictions
ecg_data[ecg_data['poss_AF_tag'] == 1]["noise_prediction"].value_counts(dropna=False)

 NaN         1792
-0.701225       2
-1.044475       2
 0.995961       2
-0.849019       2
             ... 
-0.048315       1
-0.962363       1
-0.232395       1
-0.943354       1
 0.147504       1
Name: noise_prediction, Length: 10180, dtype: int64